## Import Libraries

In [65]:
import os 
import zipfile
import tensorflow as tf

## Unzip Data

In [66]:
zip = "data.zip"
zip_ref = zipfile.ZipFile(zip,"r")
zip_ref.extractall('/data')
zip_ref.close()

zip = "validation.zip"
zip_ref = zipfile.ZipFile(zip,"r")
zip_ref.extractall("/val")
zip_ref.close()

## Divide into training and testing

In [67]:
TRAINING_DIR = "/data/data"
train_generator = tf.keras.utils.image_dataset_from_directory(TRAINING_DIR, image_size = (224,224), batch_size = 8 , shuffle = True)
VALIDATION_DIR = "/val/validation"
val_generator = tf.keras.utils.image_dataset_from_directory(VALIDATION_DIR, image_size = (224,224))

Found 883 files belonging to 3 classes.
Found 101 files belonging to 3 classes.


## Data Augmentation

In [68]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(30), 
    tf.keras.layers.RandomTranslation(0.2,0.2),
    tf.keras.layers.RandomZoom(0.4), 
    tf.keras.layers.RandomFlip("horizontal")
])

## Download MobileNetV2 pretrained model

In [69]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
IMG_SHAPE = (224,224) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

## Freeze all layers

In [70]:
base_model.trainable = False

## Add few layers on top of the base model

In [71]:
inputs = tf.keras.layers.Input((224,224,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training = False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(3, activation = 'softmax')(x)

## Compile Model

In [72]:
head_model = tf.keras.Model(inputs = inputs, outputs = predictions)
head_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

## Train the model

In [73]:
history_head = head_model.fit(train_generator, epochs = 5 , validation_data = val_generator)

Epoch 1/5


111/111 [==============================] - 22s 123ms/step - loss: 0.8018 - accuracy: 0.7508 - val_loss: 0.6478 - val_accuracy: 0.7030
Epoch 2/5
111/111 [==============================] - 7s 59ms/step - loss: 0.3099 - accuracy: 0.8822 - val_loss: 0.0893 - val_accuracy: 0.9703
Epoch 3/5
111/111 [==============================] - 7s 58ms/step - loss: 0.2425 - accuracy: 0.9241 - val_loss: 0.1892 - val_accuracy: 0.8812
Epoch 4/5
111/111 [==============================] - 7s 59ms/step - loss: 0.2087 - accuracy: 0.9343 - val_loss: 0.0388 - val_accuracy: 0.9901
Epoch 5/5
111/111 [==============================] - 8s 68ms/step - loss: 0.2138 - accuracy: 0.9366 - val_loss: 0.1591 - val_accuracy: 0.9307


## Fine Tuning

In [78]:
base_model.trainable = True

In [75]:
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False


## Model Architecture

In [83]:
head_model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 sequential_3 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_12 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 tf.math.subtract_12 (TFOpLa  (None, 224, 224, 3)      0         
 mbda)                                                           
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                          

## Reduce LR by 1/10

In [76]:
head_model.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [77]:
history_fine = head_model.fit(train_generator, epochs = 3 , validation_data = val_generator)


Epoch 1/3


111/111 [==============================] - 23s 84ms/step - loss: 0.1790 - accuracy: 0.9411 - val_loss: 0.0727 - val_accuracy: 0.9604
Epoch 2/3
111/111 [==============================] - 8s 74ms/step - loss: 0.1345 - accuracy: 0.9513 - val_loss: 0.0039 - val_accuracy: 1.0000
Epoch 3/3
111/111 [==============================] - 7s 65ms/step - loss: 0.1101 - accuracy: 0.9660 - val_loss: 0.0379 - val_accuracy: 0.9802


## Save model

In [79]:
head_model.save("model.tf")

## Convert to Tf Lite model and save

In [80]:
conv = tf.lite.TFLiteConverter.from_saved_model("model.tf")

In [81]:
tflite = conv.convert()

In [82]:
with open('model.tflite', 'wb') as f:
  f.write(tflite)